In [ ]:
import sys
print(sys.executable)

import openmm
print(openmm.__version__)

In [1]:
from openmm.app import *
from openmm import *
from openmm.unit import *
import urllib.request
import sys

# ==========================================================
# Step 1: Download Protein Structure (Villin Headpiece)
# ==========================================================
# This solves the FileNotFoundError by fetching the structure directly from RCSB PDB.
pdb_id = '1VII'
pdb_filename = 'villin.pdb'

print(f"Downloading protein structure ({pdb_id})...")
urllib.request.urlretrieve(f'https://files.rcsb.org/download/{pdb_id}.pdb', pdb_filename)
pdb = PDBFile(pdb_filename)

# Using Amber14 force field and TIP3P water model
forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')

# ==========================================================
# Step 2: Set up the Simulation System
# ==========================================================
print("Creating the simulation system...")
# NonbondedMethod=NoCutoff is used for vacuum simulation (faster for testing)
system = forcefield.createSystem(pdb.topology, nonbondedMethod=NoCutoff, constraints=HBonds)

# Integrator settings: 300K temperature, 2fs time step
integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)
simulation = Simulation(pdb.topology, system, integrator)
simulation.context.setPositions(pdb.positions)

# ==========================================================
# Step 3: Energy Minimization
# ==========================================================
print("Performing energy minimization...")
simulation.minimizeEnergy()

# ==========================================================
# Step 4: Run 100 ps Production Simulation
# ==========================================================
# 100 ps / 0.002 ps = 50,000 steps
total_steps = 50000 
record_interval = 1000 # Save a frame every 1000 steps

# Reporters: Output trajectory (PDB) and simulation data (CSV-like format to console)
simulation.reporters.append(PDBReporter('output_100ps.pdb', record_interval))
simulation.reporters.append(StateDataReporter(sys.stdout, record_interval, step=True, 
                             potentialEnergy=True, temperature=True, speed=True, progress=True, totalSteps=total_steps))

print(f"Starting 100 ps simulation ({total_steps} steps)...")
simulation.step(total_steps)

print("-" * 30)
print("✅ Task Completed Successfully!")
print(f"Files generated: {pdb_filename} (initial), output_100ps.pdb (trajectory)")

Creating the simulation system...
Performing energy minimization...
Starting 100 ps simulation (50000 steps)...
#"Progress (%)","Step","Potential Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
2.0%,1000,-2086.6204833984375,276.19840791607646,0
4.0%,2000,-2078.746955871582,287.17559830264344,1.41e+03
6.0%,3000,-2055.980781555176,308.963652612959,1.41e+03
8.0%,4000,-2217.352481842041,319.89191064852076,1.43e+03
10.0%,5000,-2009.430046081543,325.2045037139468,1.43e+03
12.0%,6000,-2236.583065032959,299.60271380727,1.44e+03
14.0%,7000,-2178.093879699707,285.4666859050418,1.43e+03
16.0%,8000,-2263.3314056396484,310.00735809246726,1.43e+03
18.0%,9000,-2217.8072814941406,317.16289822824433,1.42e+03
20.0%,10000,-2118.2843856811523,293.7305744603733,1.43e+03
22.0%,11000,-2177.109718322754,289.4238548262866,1.43e+03
24.0%,12000,-2129.2216186523438,308.98711956333995,1.42e+03
26.0%,13000,-2141.827854156494,291.15852942405354,1.43e+03
28.0%,14000,-2153.1492233276367,294.01591753916284,1.43e+0